In [1]:
import os
from dotenv import load_dotenv
import mlflow 

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor

load_dotenv()

True

In [2]:
# conectar con mlflow y minio (por http)
mlflow.set_tracking_uri("http://127.0.0.1:5000")

os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://127.0.0.1:9000"
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('ACCESS_KEY')

# Register model

In [3]:
class CustomModel(mlflow.pyfunc.PythonModel):

    def predict(self, context, model_input):
        return model_input

In [4]:
# experiment_id = create_mlflow_experiment(
#     experiment_name="model_registry",
#     artifact_location="model_registry_artifacts",
#     tags={"purpose": "learning"},
# )

mlflow.set_experiment("mlflow_tracking")

# with mlflow.start_run(run_name="model_registry") as run:
with mlflow.start_run(run_name="logging_artifacts21") as run:

    model = CustomModel()
    mlflow.pyfunc.log_model(artifact_path="custom_model", python_model=model, registered_model_name="CustomModel")
    mlflow.sklearn.log_model(artifact_path="rfr_model", sk_model=RandomForestRegressor(), registered_model_name="RandomForestRegressor")
    mlflow.sklearn.log_model(artifact_path="rft_model2", sk_model=RandomForestRegressor())   

Successfully registered model 'CustomModel'.
2024/06/19 09:43:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: CustomModel, version 1
Created version '1' of model 'CustomModel'.
Successfully registered model 'RandomForestRegressor'.
2024/06/19 09:44:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestRegressor, version 1
Created version '1' of model 'RandomForestRegressor'.


In [5]:
print('tracking uri:', mlflow.get_tracking_uri())
print('artifact uri:', mlflow.get_artifact_uri())

tracking uri: http://127.0.0.1:5000
artifact uri: s3://mlflow/3/ec0a01e3fb994111acb40246b492f97c/artifacts


In [6]:
mlflow.end_run()

# Otro ejemplo

In [7]:
# Cargar el conjunto de datos de Boston Housing
data = pd.read_csv("BostonHousing.csv")

class model_pred(mlflow.pyfunc.PythonModel):

    def __init__(self, model):
        self.model = model

    def predict(self, context, data):
        # Implement your prediction logic here
        # This example assumes data is a pandas DataFrame
        predictions = self.model.predict(data)
        return predictions

model_lr = LinearRegression()

# mlflow.autolog()

mlflow.set_experiment("mlflow_tracking")

with mlflow.start_run(run_name="logging_artifacts21") as run:

    # Separar las características y la variable objetivo
    X = data.drop("medv", axis=1)
    y = data["medv"]

    model = DecisionTreeRegressor(max_depth=5)

    mlflow.pyfunc.log_model(artifact_path="model0", python_model=model_pred(model_lr), registered_model_name="model_pyfunc")
    mlflow.sklearn.log_model(artifact_path="model1", sk_model=model, registered_model_name="model_sklearn")
    mlflow.sklearn.log_model(artifact_path="model2", sk_model=model)

    # print info about the run
    print("run_id: {}".format(run.info.run_id))

Successfully registered model 'model_pyfunc'.
2024/06/19 09:44:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: model_pyfunc, version 1
Created version '1' of model 'model_pyfunc'.
Successfully registered model 'model_sklearn'.
2024/06/19 09:44:07 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: model_sklearn, version 1
Created version '1' of model 'model_sklearn'.


run_id: 762f938020d64ad9929ad0cf99b4a31e


In [8]:
print('tracking uri:', mlflow.get_tracking_uri())
print('artifact uri:', mlflow.get_artifact_uri())

tracking uri: http://127.0.0.1:5000
artifact uri: s3://mlflow/3/3a6ebfa7b1224b44a75479e2a5bd5541/artifacts


In [9]:
mlflow.end_run()

In [10]:
print('ok_')

ok_
